In [1]:
import pandas as pd
import json
import openpyxl
import time
import os.path
import glob
import natsort
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from scipy import spatial
from preprocess import Preprocess

# algorithm parameter setting
MIN_SUPPORT = 0.2
MAX_LEN = 10
MIN_LEN = 2

# rule construction setting
MIN_TIMES = 5

# rule filter setting
K = 20

# file read setting
TRAIN = 75
TEST = 25
DATA_PATH = 'Datafile_0222/'
SHEET_NAME = 'Data_1'

# default setting
DATA_FILE_NAME = []
DATA_FILE_LIST = []


def estimate_train_file():
    '''
    Estimate the file rate of training files.
    '''

    all_train_fail_rate_result_test_file_dict = {}
    
    for a in range(TRAIN):
        cur_file = DATA_FILE_NAME[a]
        cur_path = DATA_PATH + cur_file
        print(" Estimating training file " +
                cur_file + " ")
        # estimate the fail rate of each training file
        p = Preprocess(
            file_name=cur_path,
            min_times=MIN_TIMES,
            sheetname=SHEET_NAME)

        train_fail_rate_dic = p.build_all_files()
        all_train_fail_rate_result_test_file_dict[cur_file] = train_fail_rate_dic

    return all_train_fail_rate_result_test_file_dict

def test_file_score(train_file):
    
    test_file_score_dic = {}
    for r in range(TEST):
        cur_file = DATA_FILE_NAME[r+TRAIN]
        cur_path = DATA_PATH + cur_file
        print(" Estimating testing file " +
                cur_file + " ")
        # estimate the fail rate of each training file
        p = Preprocess(
            file_name=cur_path,
            min_times=MIN_TIMES,
            sheetname=SHEET_NAME)

        test_file_dict = p.build_all_files()
        file_total_score = {}
        for file in train_file.keys(): # get one file info from train_file
            score = []
            for test_name in test_file_dict: # get one test from test file
                if test_name in train_file[file]: # check if this row in 
                    count = 0
                    min_col_num = min(len(test_file_dict[test_name]), len(train_file[file][test_name]))
                    for number in range(min_col_num):
                        if(test_file_dict[test_name][number] == train_file[file][test_name][number]):
                            count += 1
                    score.append(count /min_col_num)
                    assert ( count / min_col_num ) <= 1
                else:
                    score.append(0)
            assert len(score) == len(test_file_dict)
            file_total_score[file] = sum(score) / len(test_file_dict)
        test_file_score_dic[cur_file] = file_total_score

    print("\n  Estimation testing score Finished.  ")

    return test_file_score_dic

In [2]:
DATA_FILE_LIST = glob.glob(os.path.join(DATA_PATH, "*.xlsx"))
DATA_FILE_LIST = natsort.natsorted(DATA_FILE_LIST)
for file_path in DATA_FILE_LIST:
    DATA_FILE_NAME.append(os.path.basename(file_path))


all_train_fail_rate_result_file_dic = estimate_train_file()
all_test_score = test_file_score(all_train_fail_rate_result_file_dic)
import json
with open('data.json', 'w') as f:
    json.dump(all_test_score, f)
    

 Estimating training file testData_1.xlsx 
 Estimating training file testData_2.xlsx 
 Estimating training file testData_3.xlsx 
 Estimating training file testData_4.xlsx 
 Estimating training file testData_5.xlsx 
 Estimating training file testData_6.xlsx 
 Estimating training file testData_7.xlsx 
 Estimating training file testData_8.xlsx 
 Estimating training file testData_9.xlsx 
 Estimating training file testData_10.xlsx 
 Estimating training file testData_11.xlsx 
 Estimating training file testData_12.xlsx 
 Estimating training file testData_13.xlsx 
 Estimating training file testData_14.xlsx 
 Estimating training file testData_15.xlsx 
 Estimating training file testData_16.xlsx 
 Estimating training file testData_17.xlsx 
 Estimating training file testData_18.xlsx 
 Estimating training file testData_19.xlsx 
 Estimating training file testData_20.xlsx 
 Estimating training file testData_21.xlsx 
 Estimating training file testData_22.xlsx 
 Estimating training file testData_23.xls